In [36]:
# import dependencies
import boto3
import botocore.exceptions

import json

In [37]:
# Get the fully qualified ARN from role name
def omics_helper_get_role_arn(role_name):
    try:
        iam = boto3.resource('iam')
        role = iam.Role(role_name)
        role.load()  # calls GetRole to load attributes
    except ClientError:
        print("Couldn't get role named %s."%role_name)
        raise
    else:
        return role.arn

In [38]:
# recreates a policy
# if policy is attached to roles, it is firstly detached.
# if policy has additional versions ot the default, additional versions are deleted

def omics_helper_delete_policy(policy_name):
    try:
        # format policy ARN based on policy name
        sts_client = boto3.client('sts')
        account_id = sts_client.get_caller_identity()['Account']
        policy_arn = f'arn:aws:iam::{account_id}:policy/{policy_name}'
        
        iam_client = boto3.client('iam')
       
        # try to get policy
        policy_exists = True
        try:
            iam_client.get_policy(PolicyArn=policy_arn)
        except botocore.exceptions.ClientError as get_policy_ex:
                if get_policy_ex.response["Error"]["Code"] == "NoSuchEntity":
                    policy_exists = False
                else:
                    raise
                    
        # if policy exists, delete it            
        if policy_exists:
            # get policy versions and delete all non-default versions. 
            # Default version is deleted when policy itself is deleted.
            policy_versions_response = iam_client.list_policy_versions(PolicyArn=policy_arn)
            versions = policy_versions_response['Versions']
            for version in versions:
                version_id = version['VersionId']
                is_default = version['IsDefaultVersion']
                if not is_default:
                    iam_client.delete_policy_version(PolicyArn=policy_arn, VersionId=version_id)

            # detach all roles from policy (requirement for policy deletion)
            policy_entities = iam_client.list_entities_for_policy(PolicyArn=policy_arn, EntityFilter='Role')
            attached_roles = policy_entities['PolicyRoles']
            role_names = [role['RoleName'] for role in attached_roles]
            for role_name in role_names:
                iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
                
            #  delete policy. If it does not exist, handle error internally and continue
            iam_client.delete_policy(PolicyArn=policy_arn)
    
    except botocore.exceptions.ClientError:
        print("Could not delete policy %s."%policy_name)
        raise

In [39]:
# recreates a policy
# if policy is attached to roles, it is firstly detached.
# if policy has additional versions ot the default, additional versions are deleted

def omics_helper_recreate_policy(policy_name, policy_rights):
    try:
        # format policy ARN based on policy name
        sts_client = boto3.client('sts')
        account_id = sts_client.get_caller_identity()['Account']
        policy_arn = f'arn:aws:iam::{account_id}:policy/{policy_name}'
                
        iam_client = boto3.client('iam')
       
        # delete policy
        omics_helper_delete_policy(policy_name)
        
        # create policy
        create_policy_response = iam_client.create_policy(PolicyName=policy_name, PolicyDocument=json.dumps(policy_rights))
        
        return create_policy_response
    
    except botocore.exceptions.ClientError:
        print("Couldn't re create policy named %s."%policy_name)
        raise

In [40]:
def omics_helper_recreate_role(role_name, trust_policy):
    try:
        # format policy ARN based on policy name
        sts_client = boto3.client('sts')
        account_id = sts_client.get_caller_identity()['Account']
        
        #  delete role. If it does not exist, handle error internally and continue
        try:
            iam_client = boto3.client('iam')
            
            # delete any attached policies
            response = iam_client.list_attached_role_policies(RoleName=role_name)
            for policy in response['AttachedPolicies']:
                omics_helper_delete_policy(policy["PolicyName"])
            
            # delete role
            iam_client.delete_role(RoleName=role_name)
            
        except botocore.exceptions.ClientError as delete_ex:
            if delete_ex.response["Error"]["Code"] != "NoSuchEntity":
                raise
        
        # create role
        role = iam_client.create_role(RoleName=role_name, AssumeRolePolicyDocument=json.dumps(trust_policy))
        return role
    
    except botocore.exceptions.ClientError:
        print("Couldn't re create role named %s."%role_name)
        raise

In [41]:
def omics_helper_pretty_print(obj):
    
    # remove response metadata, if any
    if isinstance(obj, dict):
        if 'ResponseMetadata' in obj:
             del obj['ResponseMetadata']
        
    # return in json format
    return print(json.dumps(obj, indent=2 , default=str))